### Day 24 - Part 2:

- now need to account for any adjacent values

In [35]:
from collections import defaultdict

# Read in test data
filepath = "day24_test_data.txt"
with open(filepath) as fh:
    lines = [line.strip() for line in fh.readlines()]

In [36]:
def createCoords(d):
    "Input a char and output a tuple represents (x,y) change"
    
    # when we step left or right we take a much bigger step since not splitting vector in two directions
    # need to take 1/2 for each bi-directional shift (shows clearly on a visual of hex grids)
    if d == 'e':
        return (1,0)
    elif d == 'se':
        return (0.5, -0.5)
    elif d == 'sw':
        return (-0.5,-0.5)
    elif d == 'w':
        return (-1,0)
    elif d == 'nw':
        return (-0.5,0.5)
    else:
        return (0.5,0.5)

In [37]:
# This becomes a bit messier since we need to account for e vs se. Not clear consistency. 
all_coords = []
all_directions = [] # strictly for testing
flipped_dict = defaultdict(int) # track specific coords at each line & store times visited 
for line in lines:
    direction_list = []
    coord_moves = []
    i = 0
    position = [0,0]
    while i < len(line):
        
        # we see if we need to skip 2
        if line[i] in ['n', 's']:
            dir_char = line[i:i+2]
            direction_list.append(dir_char) # only for testing
            i += 2 # skip ahead 2
            coord = createCoords(dir_char)
            coord_moves.append(coord)
        else:
            dir_char = line[i]
            direction_list.append(dir_char) # only for testing
            i += 1 # only 1 step
            coord = createCoords(dir_char)
            coord_moves.append(coord)
    
        # update position 
        position[0] += coord[0]
        position[1] += coord[1]
    
    # take final position & add to default dict 
    flipped_dict[(position[0], position[1])] += 1
    del position
    # append to total list
    all_coords.append(coord_moves)
    all_directions.append(direction_list)

In [38]:
print(all_directions[0])
print(all_coords[0])

['se', 'se', 'nw', 'ne', 'ne', 'ne', 'w', 'se', 'e', 'sw', 'w', 'sw', 'sw', 'w', 'ne', 'ne', 'w', 'se', 'w', 'sw']
[(0.5, -0.5), (0.5, -0.5), (-0.5, 0.5), (0.5, 0.5), (0.5, 0.5), (0.5, 0.5), (-1, 0), (0.5, -0.5), (1, 0), (-0.5, -0.5), (-1, 0), (-0.5, -0.5), (-0.5, -0.5), (-1, 0), (0.5, 0.5), (0.5, 0.5), (-1, 0), (0.5, -0.5), (-1, 0), (-0.5, -0.5)]


In [39]:
flipped_dict

defaultdict(int,
            {(-2.0, -1.0): 1,
             (-0.5, 1.5): 2,
             (-1.5, -1.5): 1,
             (1.0, 1.0): 2,
             (0.0, 1.0): 2,
             (-1.0, 0.0): 2,
             (-2.0, 0.0): 1,
             (-0.5, 0.5): 1,
             (-1.5, -0.5): 1,
             (-1.0, 1.0): 2,
             (1.5, 1.5): 1,
             (1.0, -1.0): 1,
             (0.0, 0.0): 1,
             (2.0, 0.0): 1,
             (-1.5, 0.5): 1})

In [56]:
# Logic - Anything flipped an odd amount is black
tot_black = []
for k,v in flipped_dict.items():
    if v % 2 == 1:
        tot_black.append(k)
print(tot_black)

[(-2.0, -1.0), (-1.5, -1.5), (-2.0, 0.0), (-0.5, 0.5), (-1.5, -0.5), (1.5, 1.5), (1.0, -1.0), (0.0, 0.0), (2.0, 0.0), (-1.5, 0.5)]


#### Logic to Apply: 

- Need to determine the 6 touching hexagons. 
- Also need to identify the x, y coords of the largest hexagon:
    - I initially did this by expanding a grid but that isn't necessary.
    
    ```python
    import numpy as np
    from itertools import product
    x_vals = np.linspace(-10, 10, 41)
    y_vals = np.linspace(-10, 10, 41)

    # build out all coords:
    searchGrid = list(product(x_vals, y_vals))
```
- However, all I need to do is find the tiles that touch black at the start & return a list of those. This helps reduce the number of searches being done. 

In [76]:
def findBlackTiles(position, tot_black):
    """Check all 6 adjacent positions to see if coords exist in flipped coords"""
    
    black_count = 0
    for x,y in [createCoords(d) for d in ['e', 'se', 'sw', 'w', 'nw', 'ne']]:
        
        test_position = (position[0] + x, position[1] + y)
        if test_position in tot_black:
            black_count += 1
    
    return black_count

In [77]:
def findTilesInterest(tot_black):
    """Find any tile touching a black tile at the current state"""
    important_list = set()
    
    for position in tot_black:
        for x,y in [createCoords(d) for d in ['e', 'se', 'sw', 'w', 'nw', 'ne']]:
            important_list.add((position[0] + x, position[1] + y))
    
    return list(important_list)

In [80]:
import time
start = time.time()

# Logic - Anything flipped an odd amount is black
tot_black = []
for k,v in flipped_dict.items():
    if v % 2 == 1:
        tot_black.append(k)

for i in range(100):
    new_black_list = []
    searchGrid = findTilesInterest(tot_black)
    
    for hex_pos in searchGrid:

        # find total black tiles adjacent
        black_tiles = findBlackTiles(hex_pos, tot_black)

        if hex_pos in tot_black:
            if black_tiles == 0 or black_tiles > 2:
                pass
            else:
                new_black_list.append(hex_pos)

        else:
            if black_tiles == 2:
                new_black_list.append(hex_pos)

    # new black list overwrites the tot_black list
    print(f'Day {i+1}: {len(new_black_list)}')
    del tot_black
    tot_black = new_black_list
    del new_black_list
    
end = time.time()
print(f"Total time: {end - start}")

Day 1: 15
Day 2: 12
Day 3: 25
Day 4: 14
Day 5: 23
Day 6: 28
Day 7: 41
Day 8: 37
Day 9: 49
Day 10: 37
Day 11: 55
Day 12: 54
Day 13: 69
Day 14: 73
Day 15: 84
Day 16: 92
Day 17: 88
Day 18: 107
Day 19: 113
Day 20: 132
Day 21: 133
Day 22: 147
Day 23: 134
Day 24: 177
Day 25: 170
Day 26: 176
Day 27: 221
Day 28: 208
Day 29: 207
Day 30: 259
Day 31: 277
Day 32: 283
Day 33: 270
Day 34: 324
Day 35: 326
Day 36: 333
Day 37: 345
Day 38: 371
Day 39: 380
Day 40: 406
Day 41: 439
Day 42: 466
Day 43: 449
Day 44: 478
Day 45: 529
Day 46: 525
Day 47: 570
Day 48: 588
Day 49: 576
Day 50: 566
Day 51: 636
Day 52: 601
Day 53: 667
Day 54: 672
Day 55: 735
Day 56: 766
Day 57: 723
Day 58: 755
Day 59: 805
Day 60: 788
Day 61: 844
Day 62: 875
Day 63: 908
Day 64: 936
Day 65: 994
Day 66: 943
Day 67: 1015
Day 68: 1029
Day 69: 1058
Day 70: 1106
Day 71: 1158
Day 72: 1146
Day 73: 1125
Day 74: 1159
Day 75: 1202
Day 76: 1344
Day 77: 1277
Day 78: 1345
Day 79: 1320
Day 80: 1373
Day 81: 1420
Day 82: 1431
Day 83: 1469
Day 84: 1561


#### Appendix:

A pretty ugly grid solution below which was quite slow, but would expand and check at the endpoints of the extreme black tiles. 

In [81]:
def buildGrid(tot_black):
    """Build a grid that is slightly larger than our most extreme black hexagons"""
    min_x, min_y, max_x, max_y = 0,0,0,0
    for coord in tot_black:
        if coord[0] < min_x:
            min_x = coord[0]
        if coord[1] < min_y:
            min_y = coord[1]
        if coord[0] > max_x:
            max_x = coord[0]
        if coord[1] > max_y:
            max_y = coord[1]
    
    # ugly expansion - there are cleaner ways to handle this 
    min_x -= 1
    min_y -= 1
    max_x += 1
    max_y += 1
    
    # build our grid
    x_ticks = abs(max_x - min_x) * 2 + 1
    x_vals = np.linspace(min_x, max_x, int(x_ticks))
    y_ticks = abs(max_y - min_y) * 2 + 1
    y_vals = np.linspace(min_y, max_y, int(y_ticks))
    
    return list(product(x_vals, y_vals))